In [132]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import pymongo
import json
import censusData

********************************************
**Animal bites**
********************************************

Exploring and Cleaning dataset


In [133]:
animalBites=pd.read_csv("Resources/Health_AnimalBites.csv")
animalBites.head()

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc
0,1985-05-05 00:00:00,DOG,NaN,FEMALE,LIG. BROWN,1.0,1985-06-20 00:00:00,40229,NO,BODY,1985-05-05 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
1,1986-02-12 00:00:00,DOG,NaN,UNKNOWN,BRO & BLA,NaN,NaN,40218,NO,BODY,1986-02-12 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
2,1987-05-07 00:00:00,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
3,1988-10-02 00:00:00,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
4,1989-08-29 00:00:00,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,NaN,UNKNOWN,NaN,NaN,UNKNOWN


In [134]:
#Remove Null rows
animalBites.dropna(subset = ['bite_date','victim_zip','SpeciesIDDesc','BreedIDDesc'],inplace=True)
#Extract the year from bite_date  
animalBites['bite_year']=animalBites['bite_date'].apply(lambda x: int(x[:4]) if pd.notnull(x) else 0)
#Filter the years 2014,2015,2016,2017
animalBites=animalBites[animalBites['bite_year']>2013].reset_index(drop=True)
animalBites.head()

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,bite_year
0,2014-03-24 00:00:00,DOG,PIT BULL,NaN,WHITE,NaN,NaN,40205,NaN,BODY,NaN,NaN,2017-03-24 00:00:00,NaN,NEGATIVE,2014
1,2014-05-22 00:00:00,DOG,PIT BULL,NaN,GRAY/WHITE,NaN,NaN,40211,NaN,BODY,NaN,NaN,2014-05-22 00:00:00,NaN,NaN,2014
2,2014-12-02 00:00:00,DOG,BEAGLE,FEMALE,TRI,NaN,2014-04-01 00:00:00,40208,NaN,HEAD,NaN,NaN,2014-12-05 00:00:00,NaN,NEGATIVE,2014
3,2015-04-30 00:00:00,DOG,AAUST. TERR.,FEMALE,BROWN,NaN,NaN,40220,NO,BODY,NaN,KILLED,2015-04-30 00:00:00,NaN,NEGATIVE,2015
4,2015-05-11 00:00:00,DOG,LABRADOR RETRIV,FEMALE,GOLDEN,NaN,NaN,40047,NO,UNKNOWN,NaN,KILLED,2015-05-18 00:00:00,NaN,NEGATIVE,2015


In [135]:
columns=['year','zip','species','breed','color','gender'
         ,'wherebitten']
animalBites=animalBites.rename(columns={'bite_year':'year',
                                                 'victim_zip':'zip',
                                                 'SpeciesIDDesc':'species',
                                                 'BreedIDDesc':'breed',
                                                 'GenderIDDesc':'gender',
                                                 'WhereBittenIDDesc':'wherebitten'})

df_animalBites=animalBites[columns].copy()
df_animalBites['year']=df_animalBites['year'].astype('str')
print(df_animalBites.info())
df_animalBites.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1971 entries, 0 to 1970
Data columns (total 7 columns):
year           1971 non-null object
zip            1971 non-null object
species        1971 non-null object
breed          1971 non-null object
color          1775 non-null object
gender         1684 non-null object
wherebitten    1943 non-null object
dtypes: object(7)
memory usage: 107.9+ KB
None


,year,zip,species,breed,color,gender,wherebitten
0,2014,40205,DOG,PIT BULL,WHITE,NaN,BODY
1,2014,40211,DOG,PIT BULL,GRAY/WHITE,NaN,BODY
2,2014,40208,DOG,BEAGLE,TRI,FEMALE,HEAD
3,2015,40220,DOG,AAUST. TERR.,BROWN,FEMALE,BODY
4,2015,40047,DOG,LABRADOR RETRIV,GOLDEN,FEMALE,UNKNOWN


********************************************
**Census Data**
********************************************

Exploring and Cleaning dataset

In [136]:
df_census1=censusData.loadCensus1()
df_census1.head()

,zip,PopInPoverty,PopEmployed,MarriedPastYr,PercentFemaleEmployed,EstMeanIncAll,year
0,03450,901,786,None,-666666666.0,450,2014
1,03451,4026,3363,None,51250.0,1804,2014
2,03452,5312,4500,None,-666666666.0,2246,2014
3,03455,1864,1569,None,-666666666.0,851,2014
4,03456,753,596,None,-666666666.0,298,2014


In [6]:
df_census2=censusData.loadCensus2()
df_census2.head()

,zip,PropVal,Per Capita Income,Household Income,Population,year
0,00601,18088,7229,10833,18088,2014
1,00602,40859,9048,16353,40859,2014
2,00603,53162,9888,16323,53162,2014
3,00606,6415,6385,14138,6415,2014
4,00610,28805,8197,17265,28805,2014


********************************************
**Transform Data**
********************************************

In [137]:
# Merge Census Data
columns=['zip','year','PopInPoverty','PopEmployed','Household Income','Population']
df_census = pd.merge(df_census1,df_census2, on = ['zip','year'], how='left')[columns]
df_census.drop_duplicates(inplace=True)
df_census.head()

,zip,year,PopInPoverty,PopEmployed,Household Income,Population
0,03450,2014,901,786,55900,911
1,03451,2014,4026,3363,48382,4034
2,03452,2014,5312,4500,54420,5416
3,03455,2014,1864,1569,52695,1870
4,03456,2014,753,596,66250,753


In [50]:
# Merge Census Data with Animal Bites data
df_animal_census = pd.merge(df_animalBites,df_census, on = ['zip','year'], how='left')
print(df_animal_census.info())
df_animal_census.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2225 entries, 0 to 2224
Data columns (total 11 columns):
year                2225 non-null object
zip                 2225 non-null object
species             2225 non-null object
breed               2225 non-null object
color               1984 non-null object
gender              1900 non-null object
wherebitten         2193 non-null object
PopInPoverty        1964 non-null float64
PopEmployed         1964 non-null float64
Household Income    1964 non-null object
Population          1964 non-null float64
dtypes: float64(3), object(8)
memory usage: 208.6+ KB
None


,year,zip,species,breed,color,gender,wherebitten,PopInPoverty,PopEmployed,Household Income,Population
0,2014,40205,DOG,PIT BULL,WHITE,NaN,BODY,22589.0,20479.0,69275,24068.0
1,2014,40211,DOG,PIT BULL,GRAY/WHITE,NaN,BODY,23456.0,17243.0,25228,23622.0
2,2014,40208,DOG,BEAGLE,TRI,FEMALE,HEAD,12482.0,13609.0,25753,15480.0
3,2015,40220,DOG,AAUST. TERR.,BROWN,FEMALE,BODY,32474.0,27282.0,52155,33009.0
4,2015,40047,DOG,LABRADOR RETRIV,GOLDEN,FEMALE,UNKNOWN,20096.0,15793.0,64840,20197.0


In [153]:
# Use PyMongo to establish Mongo connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [154]:
# Define database and collection
db = client.AnimalBites
bites = db.Bites

In [166]:
#Insert all the records into bites collection
try:
    bites.drop()
    bites.insert_many(df_animal_census.to_dict('records'))
    print('Insert completed')
except Exception as e:
    print(e)

Insert completed


********************************************
**Querying Data**
********************************************

In [142]:
# Display top 10 records sort by Population 
listings = bites.find().sort([("Population",pymongo.DESCENDING)]).limit(5)
                              
for listing in listings:
    print("\n")
    print(listing)



{'_id': ObjectId('5c80887c4151573c886729cf'), 'year': '2015', 'zip': '46227', 'species': 'DOG', 'breed': 'PIT BULL', 'color': 'WHT BRN', 'gender': 'FEMALE', 'wherebitten': 'UNKNOWN', 'PopInPoverty': 55470.0, 'PopEmployed': 45552.0, 'Household Income': '36712', 'Population': 57840.0}


{'_id': ObjectId('5c80887c4151573c88672e7e'), 'year': '2017', 'zip': '43228', 'species': 'DOG', 'breed': 'PIT BULL', 'color': 'TAN WHT', 'gender': 'MALE', 'wherebitten': 'BODY', 'PopInPoverty': 54024.0, 'PopEmployed': 40739.0, 'Household Income': '43643', 'Population': 54594.0}


{'_id': ObjectId('5c80887c4151573c88672b30'), 'year': '2016', 'zip': '42701', 'species': 'DOG', 'breed': 'POODLE', 'color': 'GRY WHIT', 'gender': 'MALE', 'wherebitten': 'BODY', 'PopInPoverty': 49088.0, 'PopEmployed': 39582.0, 'Household Income': '52982', 'Population': 50533.0}


{'_id': ObjectId('5c80887c4151573c886729f9'), 'year': '2015', 'zip': '42701', 'species': 'DOG', 'breed': 'POMERANIAN', 'color': 'TAN', 'gender': 'MALE'

In [147]:
from bson.son import SON
#Number of bite incidents for zip 47172 sorted by count 
pipeline = [
    {"$match": {"zip": "47172"}},
    {"$group" : {"_id" :{"zip":"$zip","year":"$year"}, "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1)])}
]
    
listings=bites.aggregate(pipeline)

for listing in listings:
    print("\n")
    print(listing)



{'_id': {'zip': '47172', 'year': '2016'}, 'count': 2}


{'_id': {'zip': '47172', 'year': '2017'}, 'count': 1}


{'_id': {'zip': '47172', 'year': '2013'}, 'count': 1}
